In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from collections import Counter
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from string import punctuation
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sidor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sidor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
mystem = Mystem()
morph = MorphAnalyzer()

In [3]:
stops = stopwords.words('russian')
l = ['цинь', 'дэ-говать', 'юань', 'тина-фа', 'юй-фэнь', 'чжан', 'фу-цюань', 'хэ', 'цзы-сюй', 'чжоу', 'лян', 'цзин-чунь', 'дин', 'чунь-сю', 'чэнь', 'итая', 'госпожа', 'шэнь', 'ван', 'чжан', 'чжао', 'ли-мина', 'чжоу', 'бо-тао', 'чжан', 'би-сю', 'ду', 'дасинь', 'чэнь', 'чжэнь', 'чжан', 'хой-ж', 'сяо', 'шаня', 'чэн', 'цзянь-бин', 'хуан', 'цунуть-жэнить', 'ду', 'дасинить', 'лао', 'шэ', 'чэнь', 'чжэнить', 'лян', 'цзин-чуня', 'чжан', 'вэйцюня', 'цинь', 'дэ-гий', 'мэн', 'сю-ди', 'гао','это','ничто','свой','дарья', 'михайло', 'марья', 'дмитрий', 'павел', 'пётр', 'варвар', 'павло', 'марфа', 'тимофей', 'николай', 'пётр', 'валентин', 'михайло', 'александр', 'павло', 'анна', 'сергей', 'анна', 'василий', 'николай', 'артемий', 'василий', 'иван', 'увар', 'иван', 'парамон', 'семён', 'татьяна', 'борис', 'пантелей', 'еремей', 'аркадий', 'павлыч', 'фёдор', 'иван', 'евгений', 'василья', 'арин', 'власий', 'наталья', 'алексей', 'андрей', 'пётр', 'мардарий', 'аполлон', 'вера', 'николай', 'марья', 'александр', 'софья', 'николай']
stops.extend(l)

In [4]:
def ngrammer(tokens, n=10):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [5]:
#убираем знаки препинания и приводим к нижнему регистру
def normalize(text):
    punct = ",—»«][.…!?-:)("
    clean1 = [morph.parse(word.strip(punct))[0].normal_form for word in text.lower().split()]
    n = 1545
    split_text = [clean1[i:i+n] for i in range(0,len(clean1),n)]
    joined_text = [" ".join(i) for i in split_text]
    text_series = pd.Series(joined_text)
    text_df = pd.DataFrame(text_series)
    return text_df

In [2]:
import csv
import pandas as pd

In [99]:
data = pd.read_excel('Book2.xlsx')
data

,text_name,борьба_активный,борьба_антиманьчжурский,борьба_антифеодальный,борьба_беззаветный,борьба_бесконечный,борьба_беспощадный,борьба_вьетнамский,борьба_долгий,борьба_должный,...,чувство_тревожный,чувство_тяжелый,чувство_удивительный,чувство_хвалёный,чувство_хороший,чувство_человеческий,чувство_чистый,чувство_чуждый,чувство_эгоистический,category
0,Ай У. В огне рождается сталь - royallib.com.tx...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
1,Ай У. В огне рождается сталь - royallib.com.tx...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
2,Ай У. В огне рождается сталь - royallib.com.tx...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
3,Ай У. В огне рождается сталь - royallib.com.tx...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
4,Ай У. В огне рождается сталь - royallib.com.tx...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,Ба Цзинь. ХОЛОДНАЯ НОЧЬ.Роман.txt_file_27.txt_...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
476,Ба Цзинь. ХОЛОДНАЯ НОЧЬ.Роман.txt_file_28.txt_...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
477,Ба Цзинь. ХОЛОДНАЯ НОЧЬ.Роман.txt_file_29.txt_...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
478,Ба Цзинь. ХОЛОДНАЯ НОЧЬ.Роман.txt_file_30.txt_...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy


In [100]:
train_data = data.drop('text_name',axis = 1)
train_data

,борьба_активный,борьба_антиманьчжурский,борьба_антифеодальный,борьба_беззаветный,борьба_бесконечный,борьба_беспощадный,борьба_вьетнамский,борьба_долгий,борьба_должный,борьба_душевный,...,чувство_тревожный,чувство_тяжелый,чувство_удивительный,чувство_хвалёный,чувство_хороший,чувство_человеческий,чувство_чистый,чувство_чуждый,чувство_эгоистический,category
0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
2,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
3,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
4,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
476,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
477,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy
478,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,turgenevcy


In [101]:
train_data1 = train_data.drop('category',axis=1)
train_data1

,борьба_активный,борьба_антиманьчжурский,борьба_антифеодальный,борьба_беззаветный,борьба_бесконечный,борьба_беспощадный,борьба_вьетнамский,борьба_долгий,борьба_должный,борьба_душевный,...,чувство_тоскливый,чувство_тревожный,чувство_тяжелый,чувство_удивительный,чувство_хвалёный,чувство_хороший,чувство_человеческий,чувство_чистый,чувство_чуждый,чувство_эгоистический
0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
476,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
477,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
478,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
type(train_data1)

pandas.core.frame.DataFrame

In [103]:
my_list = list(train_data1.columns) 
enumerated_object = {}
for number, item in enumerate(my_list):
    enumerated_object[item] = number

In [104]:
enumerated_object

{'борьба_активный': 0,
 'борьба_антиманьчжурский': 1,
 'борьба_антифеодальный': 2,
 'борьба_беззаветный': 3,
 'борьба_бесконечный': 4,
 'борьба_беспощадный': 5,
 'борьба_вьетнамский': 6,
 'борьба_долгий': 7,
 'борьба_должный': 8,
 'борьба_душевный': 9,
 'борьба_жестокий': 10,
 'борьба_китайский': 11,
 'борьба_классовый': 12,
 'борьба_крупный': 13,
 'борьба_культурный': 14,
 'борьба_ладный': 15,
 'борьба_левый': 16,
 'борьба_межпартийный': 17,
 'борьба_минутный': 18,
 'борьба_мучительный': 19,
 'борьба_надуманный': 20,
 'борьба_наибольший': 21,
 'борьба_неистовый': 22,
 'борьба_немой': 23,
 'борьба_немыслимый': 24,
 'борьба_необходимый': 25,
 'борьба_непрерывный': 26,
 'борьба_непризнанный': 27,
 'борьба_непродолжительный': 28,
 'борьба_неравный': 29,
 'борьба_нужный': 30,
 'борьба_острый': 31,
 'борьба_отчаянный': 32,
 'борьба_партизанский': 33,
 'борьба_пассивный': 34,
 'борьба_показательный': 35,
 'борьба_политический': 36,
 'борьба_последний': 37,
 'борьба_постоянный': 38,
 'борьба_

In [105]:
vectorizer = CountVectorizer()
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [142]:
vectorizer = TfidfVectorizer()

In [106]:
#bow = vectorizer.fit_transform(df_all.combination_of_words)
X_train, X_test, y_train, y_test = train_test_split(train_data1, data.category)

In [107]:
clf = LogisticRegression()

In [108]:
clf.fit(X_train, y_train)

c:\users\sidor\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\sidor\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [109]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

     kitaycy       1.00      0.13      0.23        47
    turgenev       0.58      0.51      0.54        37
  turgenevcy       0.37      0.83      0.51        36

    accuracy                           0.46       120
   macro avg       0.65      0.49      0.43       120
weighted avg       0.68      0.46      0.41       120



In [110]:
clf.coef_

array([[ 0.        ,  0.        ,  0.        , ...,  0.00812285,
         0.11208364, -0.01037678],
       [ 0.        ,  0.        ,  0.        , ...,  0.15993541,
        -0.0540954 ,  0.02283254],
       [ 0.        ,  0.        ,  0.        , ..., -0.1693175 ,
        -0.05853626, -0.01261435]])

In [111]:
clf.coef_.shape

(3, 1253)

In [112]:
index_to_word = {
    ind: word
    for (word, ind)
    in enumerated_object.items()
    }

In [113]:
index_to_word[1095]

'человек_хороший'

In [114]:
max(clf.coef_[0])

1.3183887574023263

In [115]:
max(clf.coef_[1])

1.1362706496836281

In [116]:
max(clf.coef_[2])

1.138497480361996

In [117]:
max(enumerate(clf.coef_[1]), key=lambda pair: pair[1])

(679, 1.1362706496836281)

In [118]:
min(enumerate(clf.coef_[0]), key=lambda pair: pair[1])

(1095, -0.929532432624695)

In [125]:
top_features = sorted(enumerate(clf.coef_[0]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])
#Китайцы

человек_знаменный
человек_большой
человек_известный
женщина_свирепый
жизнь_обыденный
человек_знатный
чувство_былой
человек_смелый
человек_честный
человек_способный


In [120]:
top_features

[(726, 1.3183887574023263),
 (628, 0.9399720675687024),
 (732, 0.6136208020777637),
 (215, 0.5545384757033274),
 (377, 0.5545384757033274),
 (727, 0.5545384757033274),
 (1125, 0.5545384757033274),
 (1014, 0.5446424048983886),
 (1104, 0.5384409026530018),
 (1029, 0.47475530676329675),
 (863, 0.4572534800640855),
 (1020, 0.4488362799653745),
 (867, 0.42521065196918145),
 (787, 0.41673955825487896),
 (641, 0.4034306672547514),
 (821, 0.3527328200654227),
 (21, 0.3243302449405315),
 (645, 0.3243302449405315),
 (1113, 0.3243302449405315),
 (1208, 0.3164968717247837),
 (793, 0.30163156736619384),
 (630, 0.2975758656725954),
 (1047, 0.28323541272674874),
 (333, 0.2723912912970187),
 (474, 0.2711864934704605),
 (930, 0.26370025199289365),
 (861, 0.2372215347014078),
 (672, 0.2313511269207586),
 (354, 0.228425526175602),
 (1038, 0.22829203100108209),
 (1099, 0.22450622727102848),
 (104, 0.22273715577191236),
 (1226, 0.22102322532865293),
 (424, 0.2199855473165146),
 (800, 0.2199601706711329),
 

In [126]:
top_features = sorted(enumerate(clf.coef_[1]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])
#Тургенев

человек_дворовый
человек_практический
человек_молодой
человек_русский
человек_добрый
человек_маленький
человек_недобрый
человек_умный
человек_светский
человек_должный


In [122]:
top_features

[(679, 1.1362706496836281),
 (941, 0.9316110541761112),
 (785, 0.7710520845497165),
 (984, 0.7532038510698895),
 (689, 0.7351329228607467),
 (777, 0.7138764517586823),
 (815, 0.7040496526191959),
 (1078, 0.6848730280196125),
 (991, 0.6249939932103927),
 (691, 0.6100188924354253),
 (1188, 0.5959211319224313),
 (829, 0.5339002197771223),
 (990, 0.5339002197771223),
 (1039, 0.5339002197771223),
 (1227, 0.5339002197771223),
 (1242, 0.5339002197771223),
 (1172, 0.4986517627697199),
 (754, 0.48188738472686476),
 (728, 0.4796406490692651),
 (698, 0.4490275771188956),
 (656, 0.44575555066638023),
 (946, 0.43743912566686044),
 (1016, 0.4288070621926021),
 (671, 0.4282526412919432),
 (620, 0.37748608800155287),
 (860, 0.37396594662698746),
 (615, 0.3468462622986731),
 (975, 0.3289893296773222),
 (1115, 0.32843953800349335),
 (447, 0.3259849981495533),
 (1012, 0.3128387239617173),
 (713, 0.3012920367190054),
 (1030, 0.29437304428707856),
 (605, 0.29158966799285196),
 (247, 0.2882852358692187),
 (

In [127]:
top_features = sorted(enumerate(clf.coef_[2]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])
#Тургеневцы

человек_хороший
человек_знакомый
человек_вежливый
человек_знающий
чувство_личный
человек_несерьезный
женщина_упрямый
жизнь_хороший
человек_новый
борьба_отчаянный


In [124]:
top_features

[(1095, 1.138497480361996),
 (724, 0.5267539181268024),
 (632, 0.5241351798367714),
 (729, 0.5241351798367714),
 (1163, 0.4882647674538276),
 (851, 0.474431657565394),
 (248, 0.46119488190273755),
 (460, 0.4414320068660421),
 (865, 0.43804592870895576),
 (32, 0.4080229709921449),
 (655, 0.4007839390691691),
 (774, 0.3731152176302134),
 (1215, 0.3661952071226023),
 (1004, 0.3504786486413355),
 (368, 0.34475035351930605),
 (464, 0.3446533087924198),
 (83, 0.3073373271987499),
 (445, 0.3073373271987499),
 (1023, 0.3073373271987499),
 (533, 0.3035478774215313),
 (746, 0.2984156857883521),
 (895, 0.2980473805355324),
 (397, 0.29661358432059937),
 (120, 0.29570283294868305),
 (156, 0.29042442024266724),
 (700, 0.2888343306025088),
 (745, 0.2874374517372316),
 (1094, 0.2853889949765719),
 (765, 0.2802633979237107),
 (167, 0.27360811387372924),
 (626, 0.2729066936092268),
 (1234, 0.2559333530505175),
 (535, 0.25083036363245254),
 (246, 0.250826844408027),
 (931, 0.24744554974752295),
 (1249, 0